<a href="https://colab.research.google.com/github/MOOwuttichai/BSC_DPDM2023/blob/main/%E0%B8%A7%E0%B8%B4%E0%B9%80%E0%B8%84%E0%B8%A3%E0%B8%B2%E0%B8%B0%E0%B8%AB%E0%B9%8C%E0%B8%82%E0%B9%89%E0%B8%AD%E0%B8%A1%E0%B8%B9%E0%B8%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pythainlp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 47.8 MB/s eta 0:00:00


In [4]:
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import Tokenizer
from pythainlp.util import normalize
from pythainlp.corpus.common import thai_words
import pandas as pd

In [36]:
kok = pd.read_csv('/content/drive/MyDrive/เชี่ยงใหม่/Data_Docter_ศ.นพ.ชวลิต(หลังสกัด)_ใหม่!!.csv',encoding='utf-8')

In [37]:
kok

,name,comments,ยอดไลค์,จำนวนการตอบกลับ,จำนวนคำ,token,defind_cancer_with_nlp,use_ful,defind_exp_with_python,defind_Genden_with_nlp,...,อุจจาระเป็นเลือด,อ่อนเพลีย,เจ็บปวด,เบื่ออาหาร,เสียงแหบ,เหนื่อยง่าย,แผลเรื้อรัง,ไข้เรื้อรัง,ไม่มีการระบุอาการ,ไอเรื้อรัง
0,99แคมป์,ไม่ทานเนื้อค่ะ เห็นญาติพี่น้องรอบข้างเป็นมะ เ...,0,0,14,"['ไม่', 'ทาน', 'เนื้อ', 'ค่ะ', ' ', 'เห็น', 'ญ...",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,ไม่ระบุเพศ,...,0,0,0,0,0,0,0,0,1,0
1,A Achi Sarakam,ปี62 แม่เกร็ดเลือดต่ำ+ค่าตับผิดปกติ ตรวจพบไวรั...,0,0,96,"['ปี', '62', ' ', 'แม่', 'เกร็ด', 'เลือด', 'ต่...",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,เล่าประสบการณ์คนอื่น,เพศหญิง,...,0,0,0,0,0,0,0,0,1,0
2,A-Chiraya Kasiyapong,Bureerat Saisudta,0,0,4,"['Bureerat', ' ', 'Saisudta', 'end']",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เพศหญิง,...,0,0,0,0,0,0,0,0,1,0
3,AI'Jack TheRam,ดีมากๆเลยครับ จะได้เป็นวิทยาทานให้คนอื่น พ่อกั...,0,0,89,"['ดีมาก', 'ๆ', 'เลย', 'ครับ', ' ', 'จะ', 'ได้'...",มะเร็งเต้านม,non,เล่าประสบการณ์คนอื่น,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
4,AS Kate-Duan,.,0,0,2,"['.', 'end']",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,ไม่ว่าเจ้าไปยังที่ใด เจ้าก็ปรากฏตัวอยู่ตรงที่นั้น,กฤษมภัณฑ์ แสนเสมอ,0,0,6,"['กฤษม', 'ภัณฑ์', ' ', 'แสน', 'เสมอ', 'end']",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
1606,ไอ มิส ยู,อันนี้ไม่รู้จริงๆนะคะ เราไม่สามารถตรวจครั้งแรก...,0,1,52,"['อันนี้', 'ไม่', 'รู้', 'จริงๆ', 'นะคะ', ' ',...",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
1607,ไอด้า ไอรดาา,สมปอง มานุ,0,0,6,"['สม', 'ปอง', ' ', 'มา', 'นุ', 'end']",ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น,non,ไม่สามารถระบุได้,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,1,0
1608,杨小龙,คุณแม่พึ่งพบว่าเป็นมะเร็งรังไข่ตอนตุลาคมปีนี้ค...,0,1,84,"['คุณแม่', 'พึ่ง', 'พบ', 'ว่า', 'เป็น', 'มะเร็...",มะเร็งรังไข่,non,เล่าประสบการณ์คนอื่น,เล่าทั้งสองเพศ,...,0,0,0,0,0,0,0,0,0,0


In [38]:
len(kok[kok['defind_cancer_with_nlp']!='ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น'])

398

In [21]:
a=kok[kok['defind_cancer_with_nlp']!='ไม่สามารถระบุได้/ไม่มั่นใจว่าเป็น']

In [15]:
data = pd.read_csv('/content/drive/MyDrive/เชี่ยงใหม่/Data_Wiraon Chuenchom.csv',encoding='utf-8')

In [16]:
data

,name,comments,ยอดไลค์,จำนวนการตอบกลับ
0,ศ.นพ.ชวลิต ศูนย์มะเร็ง รพ.จุฬาลงกรณ์,โพสต์นี้ดีมีประโยชน์,677,17
1,Chewy Chillchill,หนูเป็นมะเร็งเนื้อเยื่ออ่อน ในช่องท้อง ขนาด 4 ...,3100,42
2,Joon'ly Major,แม่อายุ 51 ตรวจเจอมะเร็งเต้านมแม่มีเพื่อนเป็นม...,1100,35
3,Chanunpak Tuntisaewee,ของเราไม่มีอาการอะไรของมะเร็งเลยค่ะ ตั้งแต่ตรว...,892,30
4,Laddawan Teddy,เป็นมะเร็งหลังโพรงจมูกค่ะ อาการคือน้อยมาก แต่ค...,744,11
...,...,...,...,...
295,Chayathorn Aranphiphat,มาอ่าน ละไปตรวจร่างกายกันเถอะItsada-on K Sriva...,0,0
296,Panitan Chamsiriyanon,แม่เราอาการเริ่มแรกเบื่ออาหารมากๆกินอะไรไม่อร่...,0,0
297,Wanwisa Inted,ขอให้หายไวๆนะครับ,0,0
298,Snookkie Sa'NoOk Mak,เคสคุณตา อาการไม่ออกเลยค่ะ สำหรับมะเร็งต่อมลูก...,0,0


In [17]:
comment=data.groupby('name').sum().reset_index()
# สร้าง set ข้อมูลภาษาไทย
words = set(thai_words())
words.remove("โรคมะเร็ง")
name =['กระเพาะปัสสวะ','กระเพาะปัสสาวะ','เยื่อบุโพรงมดลูก','ปากมดลูก','เม็ดเลือดขาว','กระเพาะอาหาร','กระเพราะอาหาร','ต่อมไทรอยด์','ต่อมไทยรอยด์','ท่อน้ำดี']
for i in name:
    words.add(i)
def remove_stopthai(tokens):
    final = [word.lower()
            for word in tokens if word not in thai_stopwords()]
    return final
def remove_bark(tokens):
    final = [word.lower() for word in tokens if word not in ' ']
    return final
# สร้าง list เก็บตัว nlp เพิ่อนำไปวิเคราะห์โรค อาการ เเละเพศ
list_token =[]
count = []
for i in range(len(comment)):#len(comment)
    text= comment['comments'][i]
    custom_tokenizer = Tokenizer(words)
    Token = custom_tokenizer.word_tokenize(normalize(str(text)))
    Token.append('end')
    Token_final = remove_bark(remove_stopthai(Token))
    list_token.append(Token_final)
    count.append(len(Token_final))
comment['จำนวนคำ'] = count
comment['token'] = list_token